In [ ]:
import sys
import cv2
import glob
import os
import math
from collections import deque
from pathlib import Path

sys.path.append(str(Path.cwd().parent))
from utils.file_dialog_utils import pick_video_cv2, pick_folder

# --- PART 1: LOADING & PREPARATION ---

def load_templates_with_names(folder_path):
    """
    Loads templates and keeps their filenames.
    Returns: list of (filename, image_bgr)
    """
    paths = sorted(glob.glob(os.path.join(folder_path, "*.png")))
    template_data = []
    for p in paths:
        img = cv2.imread(p)
        if img is not None:
            filename = os.path.basename(p)
            template_data.append((filename, img))
    return template_data

def preprocess_templates_for_video(template_data):
    """
    Converts BGR templates to Grayscale ONCE to speed up video processing.
    Returns: list of (name, gray_image, width, height)
    """
    processed = []
    for name, img_bgr in template_data:
        gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
        h, w = gray.shape
        processed.append((name, gray, w, h))
    return processed

# --- PART 2: MATCHING LOGIC ---

def find_best_match_fast(target_gray, gray_templates):
    """
    Optimized matcher for video. 
    Accepts PRE-CONVERTED gray templates to save time.
    """
    best_match = {
        "score": -1.0,
        "location": None,
        "width": 0,
        "height": 0,
        "name": ""
    }

    for name, tmpl_gray, w, h in gray_templates:
        res = cv2.matchTemplate(target_gray, tmpl_gray, cv2.TM_CCOEFF_NORMED)
        _, max_val, _, max_loc = cv2.minMaxLoc(res)

        if max_val > best_match["score"]:
            best_match["score"] = max_val
            best_match["location"] = max_loc
            best_match["width"] = w
            best_match["height"] = h
            best_match["name"] = name

    return best_match

In [ ]:
# --- PART 3: VIDEO PROCESSING ---

def process_video(video_path, template_folder, output_path="matched_output.mp4"):
    # 1. Setup
    print(f"Opening video: {video_path}")
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Get video properties for the Output Writer
    fps = cap.get(cv2.CAP_PROP_FPS) or 25.0
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Setup Video Writer (Try MP4, fallback to AVI if needed)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # 2. Load and Preprocess Templates
    print("Loading and optimizing templates...")
    raw_templates = load_templates_with_names(template_folder)
    gray_templates = preprocess_templates_for_video(raw_templates)
    
    if not gray_templates:
        print("Error: No templates found.")
        return

    print(f"Starting processing on {total_frames} frames with {len(gray_templates)} templates.")
    print("-" * 50)

    # 3. Main Loop
    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break # End of video

        # Convert frame to gray
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Find best match
        result = find_best_match_fast(frame_gray, gray_templates)

        # Draw result if found
        if result['location']:
            top_left = result['location']
            bottom_right = (top_left[0] + result['width'], top_left[1] + result['height'])
            
            # Green Box
            cv2.rectangle(frame, top_left, bottom_right, (0, 255, 0), 2)
            
            # Text Label (Score)
            label = f"{result['score']:.2f}"
            cv2.putText(frame, label, (top_left[0], top_left[1]-5), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        # Write frame to output video
        writer.write(frame)

        # Progress Update (Every 10 frames to avoid spamming)
        frame_idx += 1
        if frame_idx % 10 == 0:
            print(f"Processing frame {frame_idx}/{total_frames}...")

    # 4. Cleanup
    cap.release()
    writer.release()
    print("-" * 50)
    print(f"Done! Video saved to: {output_path}")

In [ ]:
def calculate_distance(p1, p2):
    """Calculates Euclidean distance between two (x, y) points."""
    return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

def process_video_with_distance_stabilization(video_path, template_folder, output_path="matched_output.mp4", max_distance=50, min_score=0.6):
    # 1. Setup Video
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    fps = cap.get(cv2.CAP_PROP_FPS) or 25.0
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # 2. Setup Templates
    print("Loading templates...")
    raw_templates = load_templates_with_names(template_folder)
    gray_templates = preprocess_templates_for_video(raw_templates)
    
    if not gray_templates:
        print("Error: No templates found.")
        return

    # --- STABILIZATION STATE ---
    detection_history = deque(maxlen=3) 
    last_valid_location = None # Stores (x, y) of the last accepted match
    
    print(f"Starting Distance-Based Tracking...")
    print(f"Distance Threshold: {max_distance} px")
    print("Press 'q' to stop.")
    print("-" * 50)

    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break 

        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # 1. Find the raw best match for this frame
        current_match = find_best_match_fast(frame_gray, gray_templates)
        current_loc = current_match['location']
        
        # --- NEW LOGIC: DISTANCE FILTER ---
        is_good_frame = False

        if current_loc is None:
            # If matchTemplate failed completely
            is_good_frame = False
        
        elif last_valid_location is None:
            # CASE A: We are "Lost" or just starting.
            # We cannot check distance yet, so we use the Score to Lock on.
            if current_match['score'] > min_score:
                is_good_frame = True
                last_valid_location = current_loc # Initialize Lock
            else:
                is_good_frame = False
        
        else:
            # CASE B: We have a lock. Check DISTANCE.
            dist = calculate_distance(current_loc, last_valid_location)
            
            if dist < max_distance:
                # Movement is realistic
                is_good_frame = True
                last_valid_location = current_loc # Update lock to new position
            else:
                # Object teleported too far (False Positive)
                is_good_frame = False
                # NOTE: We do NOT update last_valid_location. 
                # We assume the object is still near where we last saw it.

        # 2. Update Queue
        detection_history.append(1 if is_good_frame else 0)

        # 3. Render ONLY if history is [1, 1, 1]
        if len(detection_history) == 3 and sum(detection_history) == 3:
            
            # Since the queue is full of valid matches, 'current_match' 
            # is considered the correct location.
            top_left = current_match['location']
            w, h = current_match['width'], current_match['height']
            bottom_right = (top_left[0] + w, top_left[1] + h)
            
            # Draw Green Box
            cv2.rectangle(frame, top_left, bottom_right, (0, 255, 0), 2)
            
            # Draw Label (Distance info)
            # We calculate distance just for the label display
            display_dist = 0
            if last_valid_location:
                display_dist = calculate_distance(top_left, last_valid_location)
                
            label = f"Stable | Dist: {display_dist:.1f}px"
            cv2.putText(frame, label, (top_left[0], top_left[1]-5), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
        
        # Draw a red circle where we THOUGHT the object was
        if last_valid_location:
            center_point = (last_valid_location[0] + current_match['width'] // 2,
                            last_valid_location[1] + current_match['height'] // 2)
            cv2.circle(frame, center_point, 5, (0, 0, 255), -1)

        writer.write(frame)

        # cv2.imshow("Distance Stabilization", frame)
        # if cv2.waitKey(1) & 0xFF == ord('q'):
        #     break

        frame_idx += 1
        if frame_idx % 10 == 0:
            print(f"Processing frame {frame_idx}/{total_frames}...")

    cap.release()
    writer.release()
    cv2.destroyAllWindows()
    print("-" * 50)
    print(f"Finished. Saved to: {output_path}")

In [ ]:
# --- MAIN EXECUTION ---

# Settings
_, input_video_path = pick_video_cv2(title="Select Input Video")
templates_dir = pick_folder(title="Select Template Folder")

# Run
process_video_with_distance_stabilization(input_video_path, templates_dir)